In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    StratifiedKFold
)
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer
)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier,
    VotingClassifier,
    StackingClassifier
)
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

## Data Loading

In [2]:
df = pd.read_csv("water_potability.csv")
print("First 5 rows:")
print(df.head())
print("Shape:", df.shape)

First 5 rows:
         ph    Hardness        Solids  Chloramines     Sulfate  Conductivity  \
0       NaN  204.890455  20791.318981     7.300212  368.516441    564.308654   
1  3.716080  129.422921  18630.057858     6.635246         NaN    592.885359   
2  8.099124  224.236259  19909.541732     9.275884         NaN    418.606213   
3  8.316766  214.373394  22018.417441     8.059332  356.886136    363.266516   
4  9.092223  181.101509  17978.986339     6.546600  310.135738    398.410813   

   Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       10.379783        86.990970   2.963135           0  
1       15.180013        56.329076   4.500656           0  
2       16.868637        66.420093   3.055934           0  
3       18.436524       100.341674   4.628771           0  
4       11.558279        31.997993   4.075075           0  
Shape: (3276, 10)


## Data Preprocessing

In [3]:
print("Missing values:\n", df.isnull().sum())

Missing values:
 ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64


In [4]:
def ph_category(ph):
    if pd.isna(ph):
        return np.nan
    if ph < 6.5:
        return "acidic"
    elif ph <= 8.5:
        return "neutral"
    else:
        return "alkaline"

df["ph"] = df["ph"].apply(ph_category)

In [5]:
X = df.drop('Potability',axis=1)
y = df['Potability']
numeric_features = X.select_dtypes(include = ['int64','float64']).columns
categorical_features = X.select_dtypes(include = ['object']).columns

In [6]:
def iqr_cap_outliers(X):
    X = np.asarray(X, dtype=float)
    X_capped = X.copy()

    for i in range(X.shape[1]):
        col = X[:, i]

        Q1 = np.percentile(col, 25)
        Q3 = np.percentile(col, 75)
        IQR = Q3 - Q1

        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        X_capped[:, i] = np.clip(col, lower, upper)

    return X_capped

In [7]:
num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('outlier_capper', FunctionTransformer(func=iqr_cap_outliers)),
        ('scaler', StandardScaler())
    ]
)

In [8]:
cat_transformer = Pipeline( steps = [
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
] )


In [9]:
preprocessor = ColumnTransformer(
    transformers= [
        ('num',num_transformer,numeric_features),
        ('cat',cat_transformer,categorical_features)
    ]
    )

## Pipeline creation & Primary model selection

In [10]:
clf_lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_xgb = XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

In [11]:
voting_clf = VotingClassifier(
    estimators=[
        ('lr', clf_lr),
        ('rf', clf_rf),
        ('xgb', clf_xgb)
    ],
    voting='soft'
)


In [12]:
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', clf_rf),
        ('xgb', clf_xgb)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
)


In [13]:
model_to_train = {
    'Logistic Regression': clf_lr,
    'Random Forest': clf_rf,
    'XGBoost': clf_xgb,
    'Voting Ensemble': voting_clf,
    'Stacking Ensemble': stacking_clf
}

print("""
I selected Random Forest as the primary model because:
 - the data is tabular and Random Forest handles numeric features robustly,
 - it captures non-linear relationships and feature interactions effectively,
 - it's less prone to overfitting than single decision trees (due to ensemble averaging and feature randomness),
 - it performs well on imbalanced datasets when using class weights or balanced subsampling,
 - it provides feature importance measures, aiding interpretability,
 - it requires minimal preprocessing (no feature scaling needed) and handles outliers reasonably well.

I will still evaluate other models (Logistic Regression, XGBoost, and ensembles) for comparison.
""")


I selected Random Forest as the primary model because:
 - the data is tabular and Random Forest handles numeric features robustly,
 - it captures non-linear relationships and feature interactions effectively,
 - it's less prone to overfitting than single decision trees (due to ensemble averaging and feature randomness),
 - it performs well on imbalanced datasets when using class weights or balanced subsampling,
 - it provides feature importance measures, aiding interpretability,
 - it requires minimal preprocessing (no feature scaling needed) and handles outliers reasonably well.

I will still evaluate other models (Logistic Regression, XGBoost, and ensembles) for comparison.



## Model Training

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [15]:
results = []

for name, model in model_to_train.items():

    pipe = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ]
    )

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    y_proba = pipe.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    results.append({
        "Model": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC-AUC": roc_auc
    })

results_df = pd.DataFrame(results).sort_values("ROC-AUC", ascending=False)

print(results_df)


                 Model  Accuracy  Precision    Recall  F1 Score   ROC-AUC
1        Random Forest  0.654457   0.625850  0.287500  0.394004  0.637124
4    Stacking Ensemble  0.645910   0.622951  0.237500  0.343891  0.635471
3      Voting Ensemble  0.637363   0.577181  0.268750  0.366738  0.629860
2              XGBoost  0.628816   0.559701  0.234375  0.330396  0.610696
0  Logistic Regression  0.509158   0.410088  0.584375  0.481959  0.548585


## Cross-Validation

In [16]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = []

for name, model in model_to_train.items():

    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    scores = cross_val_score(
        pipe,
        X_train,
        y_train,
        cv=cv,
        scoring='roc_auc'
    )

    cv_results.append({
        "Model": name,
        "Mean ROC-AUC": scores.mean(),
        "Std ROC-AUC": scores.std()
    })

cv_results_df = pd.DataFrame(cv_results).sort_values(
    "Mean ROC-AUC", ascending=False
)

print(cv_results_df)


                 Model  Mean ROC-AUC  Std ROC-AUC
1        Random Forest      0.660322     0.025305
4    Stacking Ensemble      0.658595     0.024681
3      Voting Ensemble      0.648947     0.025298
2              XGBoost      0.632996     0.030783
0  Logistic Regression      0.504047     0.002410


## Hyperparameter Tuning

In [17]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42))
])

rf_param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__class_weight': ['balanced']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf_grid = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=rf_param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train, y_train)

print("Best Parameters (Random Forest):")
print(rf_grid.best_params_)

print("\nBest Cross-Validated ROC-AUC:")
print(rf_grid.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters (Random Forest):
{'model__class_weight': 'balanced', 'model__max_depth': 20, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 200}

Best Cross-Validated ROC-AUC:
0.6643348584991469


## Best Model Selection

In [18]:
final_model = rf_grid.best_estimator_

print("Final Selected Model:")
print(final_model)

Final Selected Model:
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('outlier_capper',
                                                                   FunctionTransformer(func=<function iqr_cap_outliers at 0x000001A26AAA6340>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity'],
      dtype='object')),
                                                 ('cat',
                                                  Pip

## Model Performance Evaluation

In [19]:
y_test_pred = final_model.predict(X_test)
y_test_proba = final_model.predict_proba(X_test)[:, 1]

print("Final Model Test Performance:")
print("Accuracy :", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall   :", recall_score(y_test, y_test_pred))
print("F1 Score :", f1_score(y_test, y_test_pred))
print("ROC-AUC  :", roc_auc_score(y_test, y_test_proba))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

Final Model Test Performance:
Accuracy : 0.6642246642246642
Precision: 0.6256983240223464
Recall   : 0.35
F1 Score : 0.44889779559118237
ROC-AUC  : 0.6535946893787574

Confusion Matrix:
[[432  67]
 [208 112]]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.87      0.76       499
           1       0.63      0.35      0.45       320

    accuracy                           0.66       819
   macro avg       0.65      0.61      0.60       819
weighted avg       0.66      0.66      0.64       819



## Saving the final model

In [20]:
with open("final_random_forest_model.pkl", "wb") as file:
    pickle.dump(final_model, file)

print("Model saved successfully as final_random_forest_model.pkl")

Model saved successfully as final_random_forest_model.pkl
